In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import random
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, Flatten, ELU, Dropout 
from keras.optimizers import SGD
from keras.layers import Input

Using TensorFlow backend.


In [2]:
driving_log = pd.read_csv("data/driving_log.csv")

FileNotFoundError: File b'data/driving_log.csv' does not exist

In [ ]:
driving_log.head()

In [ ]:
n_train = len(driving_log)
print("Number of training examples =", n_train)

In [ ]:
central_images = driving_log["center"]
central_images.head()

In [ ]:
X = [mpimg.imread("data/" + row) for row in central_images]
y = driving_log['steering']

X = np.array(X)
y = np.array(y)

In [ ]:
index = random.randint(0, n_train)
image = X[index].squeeze()
image_dims = image.shape
output_dims = y.shape

plt.imshow(image)
print("Steering angle: {}".format(y[index]))
print("Image dimensions: {}".format(image_dims))
print("Output dimensions: {}".format(output_dims))

In [ ]:
X, y = shuffle(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
input_tensor = Input(shape=image_dims)
base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = base_model.output
x = Flatten()(x)
x = Dropout(0.2)(x)
x = ELU(x)
x = Dense(512)(x)
x = Dropout(0.5)(x)
x = ELU()(x)
predictions = Dense(1)(x)
model = Model(input=base_model.input, output=predictions)

In [ ]:
model.compile(optimizer="adam", loss="mse")


batch_size = 100
nb_epoch = 10


model.fit(X_train, y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_split=0.1)

In [ ]:
model.save_weights("model.h5")

# Save model config (architecture)
json_string = model.to_json()
with open("model.json", "w") as f:
    f.write(json_string)  